In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline

In [2]:
df = pd.read_csv('data/Train_Test_datasets/Train_Test_Network_dataset/Train_Test_Network.csv', skipinitialspace=True)

df.head()

,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1554198358,3.122.49.24,1883,192.168.1.152,52976,tcp,-,80549.530260,1762852,41933215,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
1,1554198358,192.168.1.79,47260,192.168.1.255,15600,udp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
2,1554198359,192.168.1.152,1880,192.168.1.152,51782,tcp,-,0.000000,0,0,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
3,1554198359,192.168.1.152,34296,192.168.1.152,10502,tcp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
4,1554198362,192.168.1.152,46608,192.168.1.190,53,udp,dns,0.000549,0,298,...,0,0,-,-,-,bad_UDP_checksum,-,F,0,normal


In [3]:
df.drop(columns=['ts', 'src_ip', 'src_port', 'dst_ip', 'dst_port'], inplace=True)

# for now, only work with binary classification. ignore attack type, 'type'
df.drop(columns=['type'], inplace=True)

In [4]:
df.head()

,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,...,http_request_body_len,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label
0,tcp,-,80549.530260,1762852,41933215,OTH,0,252181,14911156,2,...,0,0,0,-,-,-,bad_TCP_checksum,-,F,0
1,udp,-,0.000000,0,0,S0,0,1,63,0,...,0,0,0,-,-,-,-,-,-,0
2,tcp,-,0.000000,0,0,OTH,0,0,0,0,...,0,0,0,-,-,-,bad_TCP_checksum,-,F,0
3,tcp,-,0.000000,0,0,OTH,0,0,0,0,...,0,0,0,-,-,-,-,-,-,0
4,udp,dns,0.000549,0,298,SHR,0,0,0,2,...,0,0,0,-,-,-,bad_UDP_checksum,-,F,0


In [5]:
df.columns

Index(['proto', 'service', 'duration', 'src_bytes', 'dst_bytes', 'conn_state',
       'missed_bytes', 'src_pkts', 'src_ip_bytes', 'dst_pkts', 'dst_ip_bytes',
       'dns_query', 'dns_qclass', 'dns_qtype', 'dns_rcode', 'dns_AA', 'dns_RD',
       'dns_RA', 'dns_rejected', 'ssl_version', 'ssl_cipher', 'ssl_resumed',
       'ssl_established', 'ssl_subject', 'ssl_issuer', 'http_trans_depth',
       'http_method', 'http_uri', 'http_version', 'http_request_body_len',
       'http_response_body_len', 'http_status_code', 'http_user_agent',
       'http_orig_mime_types', 'http_resp_mime_types', 'weird_name',
       'weird_addl', 'weird_notice', 'label'],
      dtype='object')

In [6]:
df.dtypes

proto                      object
service                    object
duration                  float64
src_bytes                   int64
dst_bytes                   int64
conn_state                 object
missed_bytes                int64
src_pkts                    int64
src_ip_bytes                int64
dst_pkts                    int64
dst_ip_bytes                int64
dns_query                  object
dns_qclass                  int64
dns_qtype                   int64
dns_rcode                   int64
dns_AA                     object
dns_RD                     object
dns_RA                     object
dns_rejected               object
ssl_version                object
ssl_cipher                 object
ssl_resumed                object
ssl_established            object
ssl_subject                object
ssl_issuer                 object
http_trans_depth           object
http_method                object
http_uri                   object
http_version               object
http_request_b

In [7]:
df.weird_name.value_counts()

-                                   459749
DNS_RR_unknown_type                    753
active_connection_reuse                275
data_before_established                109
bad_UDP_checksum                        68
bad_TCP_checksum                        52
connection_originator_SYN_ack           20
above_hole_data_without_any_acks         6
inappropriate_FIN                        3
dnp3_corrupt_header_checksum             3
possible_split_routing                   3
TCP_ack_underflow_or_misorder            2
Name: weird_name, dtype: int64

In [8]:
categorical_columns = [
    'proto',
    'service',
    'conn_state',
    'dns_AA',
    'dns_RD',
    'dns_RA',
    'dns_rejected',
    'dns_query',
    'ssl_version',
    'ssl_cipher',
    'ssl_resumed',
    'ssl_established',
    'ssl_subject',
    'ssl_issuer',
    'http_trans_depth',
    'http_method',
    'http_uri',
    'http_version',
    'http_user_agent',
    'http_orig_mime_types',
    'http_resp_mime_types',
    'weird_name',
    'weird_addl',
    'weird_notice',
]
numerical_columns = set(df.columns) - set(categorical_columns)
numerical_columns

{'dns_qclass',
 'dns_qtype',
 'dns_rcode',
 'dst_bytes',
 'dst_ip_bytes',
 'dst_pkts',
 'duration',
 'http_request_body_len',
 'http_response_body_len',
 'http_status_code',
 'label',
 'missed_bytes',
 'src_bytes',
 'src_ip_bytes',
 'src_pkts'}

In [9]:
# total_unique_values = 0
# for col in categorical_columns:
#     num_unique_values = df[col].nunique()
#     print(col, '(', num_unique_values, "unique values )", '\n', df[col].value_counts(), "\n")
#     total_unique_values += num_unique_values

# print(total_unique_values, "unique categorical values")

In [10]:
# drop the dns_query column for now - it has 14372 unique values, so we can't really one-hot encode it
df.drop(columns=['dns_query'], inplace=True)

In [11]:
categorical_columns = list(set(categorical_columns) - set(['dns_query']))
print(categorical_columns)

['ssl_version', 'proto', 'weird_name', 'http_version', 'ssl_issuer', 'dns_rejected', 'http_orig_mime_types', 'dns_AA', 'ssl_resumed', 'ssl_subject', 'conn_state', 'http_user_agent', 'service', 'dns_RD', 'weird_addl', 'http_uri', 'ssl_established', 'http_trans_depth', 'http_resp_mime_types', 'ssl_cipher', 'weird_notice', 'http_method', 'dns_RA']


In [12]:
pd.get_dummies(df, columns=categorical_columns)

,duration,src_bytes,dst_bytes,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,dst_ip_bytes,dns_qclass,dns_qtype,...,ssl_cipher_TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384,weird_notice_-,weird_notice_F,http_method_-,http_method_GET,http_method_HEAD,http_method_POST,dns_RA_-,dns_RA_F,dns_RA_T
0,80549.530260,1762852,41933215,0,252181,14911156,2,236,0,0,...,0,0,1,1,0,0,0,1,0,0
1,0.000000,0,0,0,1,63,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
2,0.000000,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,1,0,0
3,0.000000,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
4,0.000549,0,298,0,0,0,2,354,0,0,...,0,0,1,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461038,0.000000,0,0,0,1,60,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
461039,0.000000,0,0,0,0,0,1,103,0,0,...,0,1,0,1,0,0,0,1,0,0
461040,290.371539,101568,2592,0,108,108064,31,3832,0,0,...,0,1,0,1,0,0,0,1,0,0
461041,23.190902,32,31,0,8,411,7,395,0,0,...,0,1,0,1,0,0,0,1,0,0


In [13]:
df[list(numerical_columns)]

,dst_bytes,dns_rcode,http_status_code,http_response_body_len,label,duration,src_bytes,missed_bytes,src_ip_bytes,dns_qtype,dst_ip_bytes,dst_pkts,dns_qclass,src_pkts,http_request_body_len
0,41933215,0,0,0,0,80549.530260,1762852,0,14911156,0,236,2,0,252181,0
1,0,0,0,0,0,0.000000,0,0,63,0,0,0,0,1,0
2,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0,0
4,298,0,0,0,0,0.000549,0,0,0,0,354,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461038,0,0,0,0,1,0.000000,0,0,60,0,0,0,0,1,0
461039,0,0,0,0,1,0.000000,0,0,0,0,103,1,0,0,0
461040,2592,0,0,0,1,290.371539,101568,0,108064,0,3832,31,0,108,0
461041,31,0,0,0,1,23.190902,32,0,411,0,395,7,0,8,0


In [17]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()
robust_scaler = RobustScaler()

df_n = df

df_n.head()

,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,...,http_request_body_len,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label
0,tcp,-,80549.530260,1762852,41933215,OTH,0,252181,14911156,2,...,0,0,0,-,-,-,bad_TCP_checksum,-,F,0
1,udp,-,0.000000,0,0,S0,0,1,63,0,...,0,0,0,-,-,-,-,-,-,0
2,tcp,-,0.000000,0,0,OTH,0,0,0,0,...,0,0,0,-,-,-,bad_TCP_checksum,-,F,0
3,tcp,-,0.000000,0,0,OTH,0,0,0,0,...,0,0,0,-,-,-,-,-,-,0
4,udp,dns,0.000549,0,298,SHR,0,0,0,2,...,0,0,0,-,-,-,bad_UDP_checksum,-,F,0


In [18]:
# df_n[list(numerical_columns)] = pd.DataFrame(standard_scaler.fit_transform(df[list(numerical_columns)]))
# df_n[list(numerical_columns)] = pd.DataFrame(robust_scaler.fit_transform(df[list(numerical_columns)]))
# df_n[list(numerical_columns)] = pd.DataFrame(min_max_scaler.fit_transform(df[list(numerical_columns)]))

In [19]:
df_n.head()

,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,...,http_request_body_len,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label
0,tcp,-,80549.530260,1762852,41933215,OTH,0,252181,14911156,2,...,0,0,0,-,-,-,bad_TCP_checksum,-,F,0
1,udp,-,0.000000,0,0,S0,0,1,63,0,...,0,0,0,-,-,-,-,-,-,0
2,tcp,-,0.000000,0,0,OTH,0,0,0,0,...,0,0,0,-,-,-,bad_TCP_checksum,-,F,0
3,tcp,-,0.000000,0,0,OTH,0,0,0,0,...,0,0,0,-,-,-,-,-,-,0
4,udp,dns,0.000549,0,298,SHR,0,0,0,2,...,0,0,0,-,-,-,bad_UDP_checksum,-,F,0


In [31]:
%%time
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from time import time
from sklearn.model_selection import train_test_split

# try 85/15 and 90/10
# make a graph to show accuracies

y = df_n.label

# cats_partial = {
#                  # 'ssl_issuer', 'http_orig_mime_types', 'ssl_cipher', 'ssl_resumed', 'http_uri',
#                  'http_user_agent',
#                 'proto',
#                 'http_resp_mime_types',
#                 'dns_rejected',
#                 'http_method',
#                  'weird_name', 'dns_RD', 'dns_AA', 'ssl_subject', 'service', 'weird_notice', 'http_version',
#                  'dns_RA', 'weird_addl', 'ssl_version',
#     #             'http_trans_depth',
#     # 'ssl_established',
#     'conn_state',
# }
cats_partial = {
                 'ssl_issuer', 'http_orig_mime_types', 'ssl_cipher', 'ssl_resumed', 'http_uri',
                 'http_user_agent',
                'proto',
                'http_resp_mime_types',
                'dns_rejected',
                'http_method',
                 'weird_name', 'dns_RD', 'dns_AA', 'ssl_subject', 'service', 'weird_notice', 'http_version',
                 'dns_RA', 'weird_addl', 'ssl_version',
                'http_trans_depth',
    'ssl_established',
    'conn_state',
}

# df_n = pd.get_dummies(df, columns=categorical_columns)
x = df_n[list(cats_partial.union(numerical_columns).difference({'label'}))]
x = pd.get_dummies(x, columns=list(cats_partial))

# tests = [.2, .15, .1]
test_size = .2
results = []

# for test_size in tests:
t1 = time()
clf = MLPClassifier(
    hidden_layer_sizes=(200),
    # max_iter=5
    # random_state=3
)

# categorical columns only
# x_categorical = x[list(set(x.columns) - numerical_columns)]

# numeric columns only
# x_numerical = x[list(numerical_columns - set(['label']))]

x_train, x_test, y_train, y_test = train_test_split(
                                                    # x_numerical,
                                                    x,
                                                    y,
                                                    # random_state=2,
                                                    # stratify=y,
                                                    test_size=test_size
                                                   )
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
t2 = time()

result = {
    'test_size': test_size,
    'accuracy': accuracy_score(y_test, y_pred),
    'conf_matrix': confusion_matrix(y_test, y_pred),
    'time': t2 - t1,
    'f1_score': f1_score(y_test, y_pred)
}
print(result)

{'test_size': 0.2, 'accuracy': 0.9189775401533473, 'conf_matrix': array([[57573,  2531],
       [ 4940, 27165]]), 'time': 75.93591475486755, 'f1_score': 0.879111988479151}
CPU times: user 5min 34s, sys: 9min 29s, total: 15min 3s
Wall time: 1min 16s


In [21]:
clf.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': 100,
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 5,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [22]:
sum(y_test)

32170

In [23]:
len(y_test)

92209

In [24]:
sum([52323,  7493, 5717, 26676])

92209

In [25]:
sum([5717, 26676])

32393

In [26]:
sum(y_pred)

38247

In [ ]:
sum(y)

In [ ]:
print(x_test.columns)

In [ ]:
for c in x_train.columns:
    print(c)